# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [55]:
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By


# navegador = webdriver.Chrome()

In [56]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
moedas = ['Dólar','Euro']
servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

dic = {}
for moeda in moedas:
    navegador.get('https://www.google.com/')
    navegador.find_element(By.XPATH,'//*[@id="APjFqb"]').send_keys(f'Cotação {moeda}')
    navegador.find_element(By.XPATH,'//*[@id="APjFqb"]').send_keys(Keys.ENTER)
    valor = navegador.find_element(
        By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
    valor = float(valor)
    dic[moeda] = valor
    #print(f"{moeda}: {valor:.2f}")

navegador.get('https://www.melhorcambio.com/ouro-hoje')    
valor = navegador.find_element(By.XPATH,'//*[@id="comercial"]').get_attribute('value')
valor = valor.replace(',','.')
dic["Ouro"] = float(valor)
#print(f"Ouro: {valor}")

navegador.quit()

### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [57]:
import pandas as pd
df = pd.read_excel('Produtos.xlsx')


- Atualizando os preços e o cálculo do Preço Final

In [58]:
df.loc[df['Moeda']=="Dólar","Cotação"] = dic['Dólar']
df.loc[df['Moeda']=="Euro","Cotação"] = dic['Euro']
df.loc[df['Moeda']=="Ouro","Cotação"] = dic['Ouro']
df['Preço de Compra'] = df['Preço Original'] * df['Cotação']
df['Preço de Venda'] = df['Preço de Compra'] * df['Margem']
df

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.4257,5425.645743,1.40,7595.904040
1,Carro Renault,4500.00,Euro,6.0550,27247.499163,2.00,54494.998326
2,Notebook Dell,899.99,Dólar,5.4257,4883.075743,1.70,8301.228763
3,IPhone,799.00,Dólar,5.4257,4335.134300,1.70,7369.728310
4,Carro Fiat,3000.00,Euro,6.0550,18164.999442,1.90,34513.498940
5,Celular Xiaomi,480.48,Dólar,5.4257,2606.940336,2.00,5213.880672
6,Joia 20g,20.00,Ouro,455.5500,9111.000000,1.15,10477.650000


### Agora vamos exportar a nova base de preços atualizada